# Hyperslicer Tutorial
John Russell September 2020

The hyperslicer function is meant to make it easier to explore imaging hyperstacks. It was designed with hyperspectral Coherent Raman Scattering in mind but should be useful for other advanced microscopy modalities such as FLIM and would play nicely with standard xyzt type stacks.

Currently this minimal version simply takes an arrary whose last two dimension are the image dimensions and all previous dimensions are taken to represent other slices of the hyperstack. It will automatically generate IntSliders that select the the relevant slice along each dimension of the input array. 

The example here uses a hyperspectral stimulated raman scattering stack of beads that are Raman active at different wavenumbers (No need to worry about the specifics but if you're interested, the experimental setup is very similar to what is described [here](https://pubs.acs.org/doi/abs/10.1021/jp308938t). You can download the dataset from [here](https://github.com/jrussell25/data-sharing/raw/master/srs_beads.npy). You can see the two different species with the larger beads (PMMA) peaking around frame 63 and the smaller beads (Polystyrene) peaking around frame 110.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
from mpl_interactions import hyperslicer
%matplotlib ipympl

In [ ]:
response = requests.get('https://github.com/jrussell25/data-sharing/raw/master/srs_beads.npy')
response.raise_for_status()
beads = np.load(io.BytesIO(response.content)) 

In [ ]:
fig1, ax1, control1 = hyperslicer(beads, vmin=0, vmax=255)

In [ ]:
beads4d = np.linspace(0,1,25)[:,None,None,None]*np.stack([beads for i in range(25)])

`beads4d` adds a linear scale from 0-1 over the intensities at each wavenumber and demonstrates the generalization to higher dimensional stacks.

In [ ]:
fig2, ax2, controls2 = hyperslicer(beads4d, vmin=0, vmax=255)

You can also provide names and or limits to map the sliders from the integers to other numbers. For instance in the `beads4d` dataset, we can replace the first dimension with the relative intensity of the image in [0,1] and the first dimension with the relevant spectrosocpic values, Raman wavenumbers. Below shows several valid ways to generate these hyperslicers.

In [ ]:
wns = np.linspace(2798.6521739130435, 3064.95652173913, beads4d.shape[1])

In [ ]:
fig3, ax3, controls3 = hyperslicer(beads4d, vmin=0, vmax=255, 
                                   axis0=(0,1), axis1=wns,
                                   names=('linear', 'wavenums')
                                   )

Instead of specifying the values for each axis and the names separately, one can use the `axes` keyword argument which expects a 2-tuple for each axis containing `(name, (start, stop))` or `(name, slider_value_array)`.

In [ ]:
fig4, ax4, controls4 = hyperslicer(beads4d, vmin=0, vmax=255, 
                                   axes=(('linear', (0,1)),('wavenums', wns))
                                   )

If you're looking for ImageJ/FIJI type experience, adding play buttons is a nice touch.

In [ ]:
fig5, ax5, controls5 = hyperslicer(beads4d, vmin=0, vmax=255, 
                                   axes=(('linear', (0,1)),('wavenums', wns)),
                                   play_buttons=True, play_button_pos='left'
                                   )

### Other ways of specifying axes

All of the below are valid calls

In [ ]:
# fig4, ax4, controls4 = hyperslicer(beads4d, vmin=0, vmax=255, 
#                                    axes=(('linear', (0,1)),('wavenums', wns))
#                                    )
# fig4, ax4, controls4 = hyperslicer(beads4d, vmin=0, vmax=255, 
#                                    axes=(('linear', (0,1)),'wavenums')
#                                    )
# fig4, ax4, controls4 = hyperslicer(beads4d, vmin=0, vmax=255, 
#                                    axes=(('linear', 0,1),'wavenums')
#                                    )
# fig4, ax4, controls4 = hyperslicer(beads4d, vmin=0, vmax=255, 
#                                    axes=((0,1),'wavenums')
#                                    )